In [ ]:
import mysql.connector
import pandas as pd
import pandas.io.sql as psql
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5)
import numpy as np
import pickle
import shutil
import re
from pathlib import Path
from collections import defaultdict

In [ ]:
target_pictures = {}
src = sorted(Path('/xxx').glob('*.dump'))
for i in src:
    with open(i, 'rb') as f:
        tmp = pickle.load(f)
        target_pictures.update(tmp)

In [ ]:
#顔が写っていない写真を抽出
src = sorted(Path('/xxx').glob('not_person*'))
for i in src:
    no_faces = [str(x.name[:-6]) for x in i.iterdir()]
    for i in list(target_pictures.keys()):
        if i[:-2] in no_faces:
            del target_pictures[i]

In [ ]:
for i in list(target_pictures.keys()):
    if target_pictures[i]["faces"][0]["attributes"]["mouthstatus"]["surgical_mask_or_respirator"] >= 90:
        del target_pictures[i]
        continue
    if target_pictures[i]["faces"][0]["attributes"]["glass"]["value"] == "Dark":
        del target_pictures[i]
        continue
    if target_pictures[i]["faces"][0]["attributes"]["eyestatus"]["left_eye_status"]["occlusion"] >= 90 or target_pictures[i]["faces"][0]["attributes"]["eyestatus"]["right_eye_status"]["occlusion"] >= 90:
        del target_pictures[i]
        continue
    if target_pictures[i]["faces"][0]["attributes"]["eyestatus"]["left_eye_status"]["no_glass_eye_close"] >= 90 or target_pictures[i]["faces"][0]["attributes"]["eyestatus"]["right_eye_status"]["no_glass_eye_close"] >= 90:
        del target_pictures[i]
        continue
    if target_pictures[i]["faces"][0]["attributes"]["eyestatus"]["left_eye_status"]["normal_glass_eye_close"] >= 90 or target_pictures[i]["faces"][0]["attributes"]["eyestatus"]["right_eye_status"]["normal_glass_eye_close"] >= 90:
        del target_pictures[i]
        continue
    if target_pictures[i]['faces'][0]['face_rectangle']['top'] <= -55:
        del target_pictures[i]
        continue

In [ ]:
pre_id = ""
score_dict = defaultdict(list)
work_dict = {}
for i, a in target_pictures.items():
    before_under_bar = i.find("_")
    id = i[:before_under_bar]
    if id != pre_id and work_dict != {}:
        #idが変わったら、値が一番大きいkeyとvalueを設定
        max_kv = max(work_dict.items(), key=lambda x: x[1])
        score_dict[max_kv[0]] = round(max_kv[1], 2)
        work_dict.clear()
    #idが変わるまで辞書に入れる
    work_dict[i] = (a['faces'][0]['attributes']['beauty']['female_score'] + a['faces'][0]['attributes']['beauty']['male_score']) / 2
    pre_id = id

In [ ]:
id_score = {}
df_list = []
for i, v in score_dict.items():
    key = re.match(r'(.*)_(.*)_(.*)', i)
    data_list = [key.group(1), key.group(2), v]
    df_list.append(data_list)

In [ ]:
id_score = pd.DataFrame(list(df_list))

In [ ]:
class Tinder:
    def __init__(self):
        self._cnx = mysql.connector.connect(user='xxx', password='xxx', database='tinder')
        self._cursor = self._cnx.cursor()

In [ ]:
t = Tinder()

In [ ]:
user = psql.read_sql("select id, city, continent from users;", t._cnx)

In [ ]:
user = user.set_index("id")
face_score = id_score.rename(columns={0: 'id', 1: 'number_photo', 2: 'score'})
face_score = face_score.set_index("id")
data = pd.concat([user, face_score], axis=1, join="inner")

In [ ]:
df_max = data.copy()
df_max["max_score"] = df_max.groupby('city')["score"].transform('max')
df_max = df_max[df_max['score'] == df_max['max_score']].sort_values("score", ascending=False)

In [ ]:
import cv2
import numpy as np

im_list = []
for i, row in df_max.iterrows():
    file_path = "/xxx" + row["city"] + "/" + i + "_" + row["number_photo"] + "_1" + ".jpg"
    im = cv2.imread(file_path)
    im_list.append(im)

In [ ]:
def concat_tile(im_list_2d):
    return cv2.vconcat([cv2.hconcat(im_list_h) for im_list_h in im_list_2d])

#同じサイズでないとタイル状に並べることができないため
im_list = [cv2.resize(x, dsize=(640, 640)) for x in im_list]
im_tile = concat_tile([[im_list[0], im_list[1], im_list[2], im_list[3]],
                       [im_list[4], im_list[5], im_list[6], im_list[7]],
                       [im_list[8], im_list[9], im_list[10], im_list[11]],
                       [im_list[12], im_list[13], im_list[14], im_list[15]],
                       [im_list[16], im_list[17], im_list[18], im_list[19]],
                       [im_list[20], im_list[21], im_list[22], im_list[23]],
                       [im_list[24], im_list[25], im_list[26], im_list[27]],
                       [im_list[28], im_list[29], im_list[30], im_list[31]],
                       [im_list[32], im_list[33], im_list[34], im_list[35]],
                       [im_list[36], im_list[37], im_list[38], im_list[39]],
                       [im_list[40], im_list[41], im_list[42], im_list[43]]])

     
cv2.imwrite('/tmp/opencv_concat_tile.jpg', im_tile)